In [1]:
import pandas as pd
import numpy as np
import matplotlib as mlp
import seaborn as sns
import plotly as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
data_v1 = pd.read_csv("C:\\Users\\magvl\\блокноты\\final project\\notebook\\data\\data_step3v1.csv")

In [5]:
data_v2 = pd.read_csv("C:\\Users\\magvl\\блокноты\\final project\\notebook\\data\\data_step3v2.csv")

In [6]:
data_v1.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,sub_area,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,43,27.0,4.0,12.558974,1.827121,1983.0,1.909804,6.399301,2.107025,15.594247,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,34,19.0,3.0,12.558974,1.827121,1958.0,1.909804,6.399301,2.107025,15.864842,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,43,29.0,2.0,12.558974,1.827121,1958.0,1.909804,6.399301,2.107025,15.613141,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,89,50.0,9.0,12.558974,1.827121,2000.0,1.909804,6.399301,2.107025,15.914449,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,77,77.0,4.0,12.558974,1.827121,1915.0,1.909804,6.399301,2.107025,16.091227,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [7]:
data_v2.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,43,27.0,4.0,12.558974,1.827121,1983.0,1.909804,6.399301,2.107025,6.407578e+06,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,34,19.0,3.0,12.558974,1.827121,1958.0,1.909804,6.399301,2.107025,9.589337e+06,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,43,29.0,2.0,12.558974,1.827121,1958.0,1.909804,6.399301,2.107025,4.808270e+06,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,89,50.0,9.0,12.558974,1.827121,2000.0,1.909804,6.399301,2.107025,1.258354e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,77,77.0,4.0,12.558974,1.827121,1915.0,1.909804,6.399301,2.107025,8.398461e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [8]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV

Возьмем эти данные за базовый вариант, прогоним на них модель.

Теперь вернемся к вопросу с временем. Так как у нас обучающая выборка с интервалом 2011-2015 года, тестовая выборка 2016 год, а предсказывать мы будем цены 2024 года и позднее, то очевидно, что это нужно учитывать. В этом нам может помочь TimeSeries.
Разбивка данных с помощью этого инструмента поможет нам учитывать влияние года реализации.

Первый вариант: one hot encoding + Target Encoding

In [9]:
X = data_v1.drop('log_price_doc', axis=1)
Y = data_v1['log_price_doc']

In [10]:
splitter = TimeSeriesSplit(n_splits=5)

In [11]:
pipe = Pipeline([('imputer', StandardScaler()), ('regression', Lasso(max_iter=100000))])

parameters = {'imputer':[StandardScaler(), MinMaxScaler()],
              'regression':[Lasso(), Ridge()],
              'regression__alpha':np.logspace(-5, 5, 11)}

select_model = GridSearchCV(pipe, parameters, scoring='neg_mean_squared_error', cv=splitter)
select_model.fit(X, Y)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=Pipeline(steps=[('imputer', StandardScaler()),
                                       ('regression', Lasso(max_iter=100000))]),
             param_grid={'imputer': [StandardScaler(), MinMaxScaler()],
                         'regression': [Lasso(), Ridge(alpha=10.0)],
                         'regression__alpha': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='neg_mean_squared_error')

In [12]:
print(select_model.best_estimator_)

Pipeline(steps=[('imputer', MinMaxScaler()), ('regression', Ridge(alpha=10.0))])


In [13]:
pipe = Pipeline([('scaler', select_model.best_params_['imputer']), 
                 ('regression', select_model.best_params_['regression'])])

In [14]:
cv_result_pipe = cross_validate(pipe, X, Y, 
                                scoring='neg_mean_squared_error',
                                cv=splitter, return_train_score=True)

print(f"Среднее MSLE на тренировочных фолдах: {-np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Среднее MSLE на тестовых фолдах: {-np.mean(cv_result_pipe['test_score']).round(3)}")

Среднее MSLE на тренировочных фолдах: 0.295
Среднее MSLE на тестовых фолдах: 0.256


Для сравнения посмотрим на Lasso

In [15]:
pipe_lasso = Pipeline([('imputer', StandardScaler()), ('regression', Lasso(max_iter=100000))])

parameters_lasso = {'imputer':[StandardScaler(), MinMaxScaler()],
                    'regression':[Lasso()],
                    'regression__alpha':np.logspace(-5, 5, 11)}

select_model_lasso = GridSearchCV(pipe_lasso, parameters_lasso, scoring='neg_mean_squared_error', cv=splitter)
select_model_lasso.fit(X, Y)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=Pipeline(steps=[('imputer', StandardScaler()),
                                       ('regression', Lasso(max_iter=100000))]),
             param_grid={'imputer': [StandardScaler(), MinMaxScaler()],
                         'regression': [Lasso(alpha=0.001)],
                         'regression__alpha': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='neg_mean_squared_error')

In [16]:
print(select_model_lasso.best_estimator_)

Pipeline(steps=[('imputer', MinMaxScaler()),
                ('regression', Lasso(alpha=0.001))])


In [17]:
pipe_lasso = Pipeline([('scaler', select_model_lasso.best_params_['imputer']), 
                 ('regression', select_model_lasso.best_params_['regression'])])

In [18]:
cv_result_pipe_lasso = cross_validate(pipe_lasso, X, Y, 
                                    scoring='neg_mean_squared_error',
                                    cv=splitter, return_train_score=True)

print(f"Среднее MSLE на тренировочных фолдах: {-np.mean(cv_result_pipe_lasso['train_score']).round(3)}")
print(f"Среднее MSLE на тестовых фолдах: {-np.mean(cv_result_pipe_lasso['test_score']).round(3)}")

Среднее MSLE на тренировочных фолдах: 0.298
Среднее MSLE на тестовых фолдах: 0.293


Посмотрим, какие признаки нам оставила модель

In [19]:
res = []
for i in range (len(X.columns)):
    if select_model_lasso.best_estimator_['regression'].coef_[i] != 0:
        res.append(X.columns[i])

In [20]:
res

['floor',
 'max_floor',
 'build_year',
 'num_room',
 'sub_area',
 'raion_popul',
 'healthcare_centers_raion',
 'additional_education_raion',
 'build_count_wood',
 'green_zone_km',
 'cemetery_km',
 'railroad_station_avto_min',
 'market_shop_km',
 'workplaces_km',
 'ecology',
 'big_church_count_500',
 'prom_part_1000',
 'trc_count_1000',
 'trc_sqm_1500',
 'mosque_count_1500',
 'cafe_sum_2000_min_price_avg',
 'mosque_count_3000',
 'sport_count_3000',
 'prom_part_5000',
 'trc_sqm_5000',
 'mosque_count_5000',
 'market_count_5000',
 'usdrub',
 'salary',
 'product_type_OwnerOccupier',
 'culture_objects_top_25_yes',
 'incineration_raion_yes',
 'radiation_raion_yes',
 'railroad_1line_yes',
 'year_2014',
 'month_2',
 'month_9',
 'month_12']

Итого в базовой версии мы имеем следующее: Lasso модель показала чуть хуже результат, чем Ridge (на 0,017 или 5%), при этом модель использовала всего 3 признака: общую площадь, количество комнат и район. Выбор признаков звучит достаточно логично. Учитывая, что одно из условий задания максимально сократить количество используемых признаков, скорее всего свой выбор мы остановим на модели Lasso. Однако, это только базовая версия, поработаем дальше еще немного над данными.

Теперь второй вариант: только one hot encoding

In [21]:
X = data_v2.drop('log_price_doc', axis=1)
Y = data_v2['log_price_doc']

In [22]:
splitter = TimeSeriesSplit(n_splits=5)

In [23]:
pipe = Pipeline([('imputer', StandardScaler()), ('regression', Lasso(max_iter=100000))])

parameters = {'imputer':[StandardScaler(), MinMaxScaler()],
              'regression':[Lasso(), Ridge()],
              'regression__alpha':np.logspace(-5, 5, 11)}

select_model = GridSearchCV(pipe, parameters, scoring='neg_mean_squared_error', cv=splitter)
select_model.fit(X, Y)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=Pipeline(steps=[('imputer', StandardScaler()),
                                       ('regression', Lasso(max_iter=100000))]),
             param_grid={'imputer': [StandardScaler(), MinMaxScaler()],
                         'regression': [Lasso(), Ridge(alpha=10.0)],
                         'regression__alpha': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='neg_mean_squared_error')

In [24]:
print(select_model.best_estimator_)

Pipeline(steps=[('imputer', MinMaxScaler()), ('regression', Ridge(alpha=10.0))])


In [25]:
pipe = Pipeline([('scaler', select_model.best_params_['imputer']), 
                 ('regression', select_model.best_params_['regression'])])

In [26]:
cv_result_pipe = cross_validate(pipe, X, Y, 
                                scoring='neg_mean_squared_error',
                                cv=splitter, return_train_score=True)

print(f"Среднее MSLE на тренировочных фолдах: {-np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Среднее MSLE на тестовых фолдах: {-np.mean(cv_result_pipe['test_score']).round(3)}")

Среднее MSLE на тренировочных фолдах: 0.291
Среднее MSLE на тестовых фолдах: 0.257


Для сравнения посмотрим на Lasso

In [27]:
pipe_lasso = Pipeline([('imputer', StandardScaler()), ('regression', Lasso(max_iter=100000))])

parameters_lasso = {'imputer':[StandardScaler(), MinMaxScaler()],
                    'regression':[Lasso()],
                    'regression__alpha':np.logspace(-5, 5, 11)}

select_model_lasso = GridSearchCV(pipe_lasso, parameters_lasso, scoring='neg_mean_squared_error', cv=splitter)
select_model_lasso.fit(X, Y)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=Pipeline(steps=[('imputer', StandardScaler()),
                                       ('regression', Lasso(max_iter=100000))]),
             param_grid={'imputer': [StandardScaler(), MinMaxScaler()],
                         'regression': [Lasso(alpha=0.001)],
                         'regression__alpha': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='neg_mean_squared_error')

In [28]:
print(select_model_lasso.best_estimator_)

Pipeline(steps=[('imputer', MinMaxScaler()),
                ('regression', Lasso(alpha=0.001))])


In [29]:
pipe_lasso = Pipeline([('scaler', select_model_lasso.best_params_['imputer']), 
                 ('regression', select_model_lasso.best_params_['regression'])])

In [30]:
cv_result_pipe_lasso = cross_validate(pipe_lasso, X, Y, 
                                    scoring='neg_mean_squared_error',
                                    cv=splitter, return_train_score=True)

print(f"Среднее MSLE на тренировочных фолдах: {-np.mean(cv_result_pipe_lasso['train_score']).round(3)}")
print(f"Среднее MSLE на тестовых фолдах: {-np.mean(cv_result_pipe_lasso['test_score']).round(3)}")

Среднее MSLE на тренировочных фолдах: 0.3
Среднее MSLE на тестовых фолдах: 0.304


In [31]:
res = []
for i in range (len(X.columns)):
    if select_model_lasso.best_estimator_['regression'].coef_[i] != 0:
        res.append(X.columns[i])

In [32]:
res

['floor',
 'max_floor',
 'material',
 'build_year',
 'num_room',
 'indust_part',
 'preschool_quota',
 'preschool_education_centers_raion',
 'school_education_centers_top_20_raion',
 'hospital_beds_raion',
 'healthcare_centers_raion',
 'build_count_block',
 'build_count_monolith',
 'build_count_panel',
 'build_count_1971-1995',
 'green_zone_km',
 'ttk_km',
 'big_road1_km',
 'nuclear_reactor_km',
 'big_market_km',
 'office_km',
 'big_church_count_500',
 'sport_count_500',
 'prom_part_1000',
 'trc_count_1000',
 'market_count_1000',
 'mosque_count_1500',
 'prom_part_2000',
 'cafe_sum_2000_min_price_avg',
 'mosque_count_3000',
 'sport_count_3000',
 'office_count_5000',
 'mosque_count_5000',
 'usdrub',
 'salary',
 'product_type_OwnerOccupier',
 'sub_area_Birjulevo Vostochnoe',
 'sub_area_Hamovniki',
 'sub_area_Krjukovo',
 'sub_area_Krylatskoe',
 'sub_area_Lomonosovskoe',
 'sub_area_Nekrasovka',
 'sub_area_Poselenie Filimonkovskoe',
 'sub_area_Poselenie Novofedorovskoe',
 'sub_area_Poselenie 

Какие мы можем сделать выводы: 

во-первых, глобальной разницы между различными вариантами не наблюдается;

во-вторых, наилучший результат на тесте показала Ridge модель с one hot encoding всех признаков - 0,281;

в-третьих, для наших задач нам удобнее будет использовать сочетание OHE и TE категориальных признаков, в этом случае модель Lasso показала результат 0,297, что на 5% хуже. Но свой выбор на текущий момент мы остановим именно на этой модели.

Да, мы несколько потеряем в качестве, но решаем сразу две проблемы: нам будет удобнее делать наш сервис с TE признака sub_area, а также модель оставила всего 3 признака, что решает нашу задачу с минимизацией признаков.